<a href="https://colab.research.google.com/github/Chiamakac/TRAININGS/blob/main/TRAININGS/IgboBERT%202.0/50EPOCH_of_Fine_tuned_IgboBert_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Token Classification (PyTorch)-Fine-Tunning 

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
!pip install datasets transformers[sentencepiece]
#!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 5.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-

You will need to setup git, adapt your email and name in the following cell.

In [2]:
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [3]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from datasets import concatenate_datasets, load_dataset, DatasetDict
ibo1 = load_dataset('masakhaner', 'ibo')
ibo2 = load_dataset('masakhane/masakhaner2', 'ibo')



Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2235 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/320 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/638 [00:00<?, ? examples/s]

Dataset masakhaner downloaded and prepared to /root/.cache/huggingface/datasets/masakhaner/ibo/1.0.0/e61b24903076a3af7682855beebb820ec64edad0d6787b148c473694592d10b3. Subsequent calls will reuse this data.


...

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset masakhaner2 downloaded and prepared to /root/.cache/huggingface/datasets/masakhane___masakhaner2/ibo/1.0.0/37ca2f236794aa08d52f7cbcdb2fb6e8df029aecfc0a4f798d04df8c342b9d3c. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
merged_dataset = []
for split in ['train','test', 'validation']:
  ibo1_split = load_dataset('masakhaner', 'ibo', split=split) 
  ibo2_split = load_dataset('masakhane/masakhaner2', 'ibo', split=split) 
  merged_dataset.append(concatenate_datasets([ibo1_split, ibo2_split]))
merged_dataset = DatasetDict({'train':merged_dataset[0], 'validation':merged_dataset[1], 'test':merged_dataset[2]})

In [ ]:
merged_dataset

In [ ]:
# displays the first element of the training set
merged_dataset["train"][0]["tokens"]

In [8]:
#displays the features attribute of our dataset
ner_feature = merged_dataset["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE'], id=None), length=-1, id=None)

In [9]:
#we can access the list of names in the ner_feature by looking at the names attribute of that feature
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE']

In [10]:
#decoding the labels we saw earlier and prints line 1 and line 2 of the training set with the labels
words = merged_dataset["train"][0]["tokens"]
labels = merged_dataset["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

Ike ịda jụụ otụ    nkeji  banyere oke ogbugbu na - eme n'ala Naijiria agwụla Ekweremmadụ 
O   O   O   B-DATE I-DATE O       O   O       O  O O   O     B-LOC    O      B-PER       


In [11]:
#mount the gdrive
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
#model path in the gdrive
MODEL_PATH = "/content/gdrive/MyDrive/IboBERT_2.0/IgboBert_2.0" #replace with the model path you want to work with

**Processing the data**

In [13]:
#we will be using a IgboBERT pretrained model.
#Download and cache the associated tokenizer

#You can replace the model_checkpoint with any other model you prefer from the Hub, 
#or with a local folder in which you’ve saved a pretrained model and a tokenizer
from transformers import AutoTokenizer

model_checkpoint = MODEL_PATH
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,add_prefix_space=True)

In [ ]:
#Tokenizing our pre-tokenized input with our tokenizer
inputs = tokenizer(merged_dataset["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

In [15]:
# Expanding our label list to match the tokens and assigning a label of -100 to special tokens 
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [16]:
# trying out the above line of code on our first sentence to see if -100 was assigned to the special tokens
labels = merged_dataset["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1]
[-100, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, -100]


In [17]:
#we tokenize all the inputs and apply align_labels_with_tokens() on all the labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [18]:
#applying all that preprocessing in one go on the other splits of our dataset
tokenized_datasets = merged_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=merged_dataset["train"].column_names,
)

  0%|          | 0/10 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

**Fine-tuning the model with the Trainer API**

In [19]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [20]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    0,    0,    0,    7,    8,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    5,    0,    1, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    5,    0,    1,
            2,    0,    0,    0,    0,    0,    0,    0,    0,    7,    8,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            5,    0,    0,    0,    0, -100]])


To have the Trainer compute a metric every epoch, we will need to define a compute_metrics() function that takes the arrays of predictions and    labels, and returns a dictionary with the metric names and values.To use this metric, we first need to install the seqeval library.

In [21]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=f4269cc115ce44d979a39c8184a6a6c8d737b206b7b0bf86c196ad89313807e7
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [22]:
from datasets import load_metric

metric = load_metric("seqeval")

<ipython-input-22-edef154daec7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [23]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [24]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [25]:
#defining the model we want to finetune
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at /content/gdrive/MyDrive/IboBERT_2.0/IgboBert_2.0 were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at /content/gdrive/MyDrive/IboBERT_2.0/IgboBert_2.0 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRA

In [26]:
#log in to Hugging Face 
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [27]:
#we define our TrainingArguments
from transformers import TrainingArguments

args = TrainingArguments(
    "IgboBert-finetuned-ner-merged",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    num_train_epochs=50,
    weight_decay=0.01,
    #push_to_hub=True, (uncomment if you want to upload your results to the Model Hub)
)

In [28]:
#we pass everything to the Trainer and start training
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9869
  Num Epochs = 50
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 61700
  Number of trainable parameters = 82867209


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.115700,0.118815,0.662670,0.762670,0.709162,0.965211
2,0.062300,0.110562,0.708604,0.765837,0.736110,0.971078
3,0.038100,0.133793,0.714522,0.780317,0.745972,0.971160
4,0.026100,0.125227,0.757804,0.774434,0.766029,0.972007
5,0.019300,0.166377,0.693777,0.764253,0.727312,0.969725
6,0.016000,0.177072,0.761470,0.773529,0.767452,0.972844
7,0.015600,0.165197,0.761894,0.793439,0.777347,0.974187
8,0.012400,0.198645,0.745325,0.757466,0.751346,0.970365
9,0.008900,0.217582,0.777195,0.771041,0.774106,0.972472
10,0.008200,0.216351,0.734446,0.766516,0.750138,0.970757


***** Running Evaluation *****
  Num examples = 2819
  Batch size = 8
Saving model checkpoint to IgboBert-finetuned-ner-merged/checkpoint-1234
Configuration saved in IgboBert-finetuned-ner-merged/checkpoint-1234/config.json
Model weights saved in IgboBert-finetuned-ner-merged/checkpoint-1234/pytorch_model.bin
tokenizer config file saved in IgboBert-finetuned-ner-merged/checkpoint-1234/tokenizer_config.json
Special tokens file saved in IgboBert-finetuned-ner-merged/checkpoint-1234/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2819
  Batch size = 8
Saving model checkpoint to IgboBert-finetuned-ner-merged/checkpoint-2468
Configuration saved in IgboBert-finetuned-ner-merged/checkpoint-2468/config.json
Model weights saved in IgboBert-finetuned-ner-merged/checkpoint-2468/pytorch_model.bin
tokenizer config file saved in IgboBert-finetuned-ner-merged/checkpoint-2468/tokenizer_config.json
Special tokens file saved in IgboBert-finetuned-ner-merged/checkpoint-2468/specia

TrainOutput(global_step=61700, training_loss=0.00852420635494242, metrics={'train_runtime': 5181.4725, 'train_samples_per_second': 95.234, 'train_steps_per_second': 11.908, 'total_flos': 7816700282650938.0, 'train_loss': 0.00852420635494242, 'epoch': 50.0})

**Using the fine-tuned model**

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "/content/IgboBert-finetuned-ner/checkpoint-5600"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("Google chetara ụbọchị ọmụmụ Keshi mgbe Obasanjo chịrị Naịjirịa afọ asatọ na ọchịchị onye kwuo uche ya."
                 "Onyeisi ndị na-emenyu ọkụ na, Legọọsi steeti bụ Rasak Fadipe ekwuola na onweghi onye nwụrụ n'ime ọkụ ahụ gbara n'ehihie ụbọchị 24 Jenuwarị, 2018.")

loading configuration file /content/IgboBert-finetuned-ner/checkpoint-5600/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/IgboBert-finetuned-ner/checkpoint-5600",
  "architectures": [
    "RobertaForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-DATE",
    "8": "I-DATE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-DATE": "7",
    "B-LOC": "5",
    "B-ORG": "3",
    "B-PER": "1",
    "I-DATE": "8",
    "I-LOC": "6",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "p

[{'end': 6,
  'entity_group': 'ORG',
  'score': 0.99998724,
  'start': 0,
  'word': ' Google'},
 {'end': 21,
  'entity_group': 'DATE',
  'score': 0.99948347,
  'start': 15,
  'word': ' ụbọchị'},
 {'end': 33,
  'entity_group': 'PER',
  'score': 0.99567986,
  'start': 28,
  'word': ' Keshi'},
 {'end': 47,
  'entity_group': 'PER',
  'score': 0.99999,
  'start': 39,
  'word': ' Obasanjo'},
 {'end': 62,
  'entity_group': 'LOC',
  'score': 0.9999815,
  'start': 54,
  'word': ' Naịjirịa'},
 {'end': 72,
  'entity_group': 'DATE',
  'score': 0.9996765,
  'start': 63,
  'word': ' afọ asatọ'},
 {'end': 146,
  'entity_group': 'LOC',
  'score': 0.99975836,
  'start': 132,
  'word': ' Legọọsi steeti'},
 {'end': 162,
  'entity_group': 'PER',
  'score': 0.99998444,
  'start': 150,
  'word': ' Rasak Fadipe'},
 {'end': 241,
  'entity_group': 'DATE',
  'score': 0.98287994,
  'start': 214,
  'word': "'ehihie ụbọchị 24 Jenuwarị,"},
 {'end': 246,
  'entity_group': 'DATE',
  'score': 0.99987113,
  'start': 24

In [ ]:
#save the fine-tuned model in gdrive
import shutil
shutil.move('/content/IgboBert-finetuned-ner-merged','/content/gdrive/MyDrive/IboBERT_2.0/FINE TUNING/mearged')

'/content/gdrive/MyDrive/IboBERT_2.0/FINE TUNING/mearged/IgboBert-finetuned-ner-merged'

# **END OF CODE**

In [ ]:
from datasets import concatenate_datasets, load_dataset 
ibo2 = load_dataset('masakhane/masakhaner2', 'ibo', split='test') 
ibo1 = load_dataset('masakhaner', 'ibo', split='test')
test = concatenate_datasets([ibo2, ibo1])



In [ ]:
test

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 2819
})

In [ ]:
from datasets import concatenate_datasets, load_dataset
ibo2 = load_dataset('masakhane/masakhaner2', 'ibo', split='validation')
ibo1 = load_dataset('masakhaner', 'ibo', split='validation')
validation = concatenate_datasets([ibo2, ibo1])

In [ ]:
from datasets import concatenate_datasets, load_dataset
ibo1 = load_dataset('masakhaner', 'ibo', split='train')
ibo2 = load_dataset('masakhane/masakhaner2', 'ibo', split='train')

In [ ]:
# from datasets import load_dataset

# url = "https://github.com/Chiamakac/TRAININGS/tree/main/IgboBERT%202.0/ibo%20merged/"
# data_files = {"dev": f"{url}dev.txt", "test": f"{url}test.txt", "train": f"{url}train.txt"}
# raw_datasets = load_dataset("text", data_files=data_files)

In [ ]:
from datasets import load_dataset

url = "https://github.com/Chiamakac/TRAININGS/tree/main/IgboBERT%202.0/ibo%20merged/"
data_files = {
    "train": url + "train.txt",
    "test": url + "test.txt",
    "dev": url + "dev.txt",
}
raw_datasets = load_dataset("text", data_files=data_files)